# Homework 5

## Description

### Data
[Google web graph dataset](http://snap.stanford.edu/data/web-Google.html) - about 5 million edwges, 20MB (compressed) in size

Nodes represent web pages and directed edges represent hyperlinks between them. 
The data was released in 2002 by Google as a part of Google Programming Contest.

### Format
One text file consisting of lines of records.

Each line is a directed edge representing hyperlinks between nodes (web pages) </br>
Example: <FromNodeID\> <ToNodeID\>

### Task
3 subtasks:
+ (30pt) Given the Google web graph dataset, please output the list of web pages with the number of outlinks, sorted in descending order of the out-degrees.
+ (30pt) Please output the inlink distribution of the top linked web pages, sorted in descending order of the in-degrees.
+ (40pt) Design an algorithm that maintains the connectivity of two nodes in an efficient way. Given a node v, please output the list of nodes that v points to, and the list of nodes that points to v.
+ (Bonus) Compute the PageRank of the graph using MapReduce.

### Output Format
1. a sorted list of pages with their out-degrees
    + Each line contains: <NodeID\>, <out-degree\>
1. a sorted list of pages with their in-degrees
    + Each line contains: <NodeID\>, <in-degree\>
1. Given a node v,
    + The first line contains a list of nodes that v points to:
        + <ToNodeID\>, …, <ToNodeID\>
    + The second line contains a list of nodes point to v
        + <FromNodeID\>, …, <FromNodeID\>
1. The PageRank of each node in steady state
    + Each line contains: <NodeID\>, <Score\>

## Implementation

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.hadoop.fs._
import org.apache.hadoop.conf.Configuration

import java.io.{File,PrintWriter}

  def printSample(writer: Any, data: Any, title: String = "", format: String = ""){
    println("\n"+title+" Data Sample: " + format)
    println(data+"\n")
  }

  def printSpark(writer: Any, spark: SparkSession): Unit = {
    println("Spark Entity:       " + spark)
    println("Spark version:      " + spark.version)
    println("Spark master:       " + spark.sparkContext.master)
    println("Running 'locally'?: " + spark.sparkContext.isLocal)
    println("")
  }

  def outputWriter(fileString: String): PrintWriter ={
    val outputPath = new Path(fileString)
    val outputStream = outputPath.getFileSystem(new Configuration()).create(outputPath);
    new PrintWriter(outputStream)
  }

  def getFile(fileString: String): Array[String] ={
    val inputPath = new Path(fileString)
    val inputBuffer = scala.collection.mutable.ArrayBuffer.empty[String]
    val iterator = inputPath.getFileSystem(new Configuration()).listFiles(inputPath, false)
    while(iterator.hasNext()){
        val fileStatus = iterator.next()
        if(fileStatus.isFile()){
          inputBuffer += fileStatus.getPath().toString()
        }
    }
    inputBuffer.toArray
  }


val writer = null
printSpark(writer, spark)

Spark Entity:       org.apache.spark.sql.SparkSession@1428ad24
Spark version:      2.3.0
Spark master:       local[*]
Running 'locally'?: true



printSample: (writer: Any, data: Any, title: String, format: String)Unit
printSpark: (writer: Any, spark: org.apache.spark.sql.SparkSession)Unit
outputWriter: (fileString: String)java.io.PrintWriter
getFile: (fileString: String)Array[String]
writer: Null = null


In [2]:
val data = spark.sparkContext.textFile("./data/data.txt")
data.take(10).foreach(println)

# Directed graph (each unordered pair of nodes is saved once): web-Google.txt 
# Webgraph from the Google programming contest, 2002
# Nodes: 875713 Edges: 5105039
# FromNodeId	ToNodeId
0	11342
0	824020
0	867923
0	891835
11342	0
11342	27469


data = ./data/data.txt MapPartitionsRDD[1] at textFile at <console>:35


./data/data.txt MapPartitionsRDD[1] at textFile at <console>:35

In [3]:
// Remove Header
val rows = data.filter(l => l.charAt(0) != '#')
rows.take(5).foreach(println)
println("Count: "+rows.count)

0	11342
0	824020
0	867923
0	891835
11342	0
Count: 5105039


rows = MapPartitionsRDD[2] at filter at <console>:38


MapPartitionsRDD[2] at filter at <console>:38

In [4]:
val flattenData = rows.
    flatMap{ dataString =>
        dataString.split("\\s+").
            zipWithIndex.
            map{
                case (value,index) => 
                    index match{
                        case 0 => (value, (0, 1))
                        case 1 => (value, (1, 0))
                    }
                }
    }.reduceByKey{ case((in1, out1), (in2, out2)) => (in1+in2, out1+out2)}
flattenData.take(10).foreach(println)

(729609,(3,4))
(28995,(25,15))
(892689,(4,4))
(393372,(2,6))
(581541,(2,4))
(700977,(0,1))
(781047,(14,14))
(617493,(3,3))
(432234,(2,2))
(899514,(21,11))


flattenData = ShuffledRDD[4] at reduceByKey at <console>:50


ShuffledRDD[4] at reduceByKey at <console>:50

### Task 1 - Find Out Degrees

In [5]:
val out = flattenData.sortBy{case(id, (in, out)) => -out}.map{case(id, (in, out)) => id+", "+out}
out.take(10).foreach(println)
out.saveAsTextFile("./output/outDegree")

506742, 456
305229, 372
203748, 372
768091, 330
808643, 277
412410, 268
600479, 265
376428, 258
156950, 257
885728, 256


out = MapPartitionsRDD[10] at map at <console>:41


MapPartitionsRDD[10] at map at <console>:41

### Task 2 - Find In Degrees

In [6]:
val in = flattenData.sortBy{case(id, (in, out)) => -in}.map{case(id, (in, out)) => id+", "+in}
in.take(10).foreach(println)
in.saveAsTextFile("./output/inDegree")

537039, 6326
597621, 5354
504140, 5271
751384, 5182
32163, 5097
885605, 4847
163075, 4731
819223, 4620
605856, 4550
828963, 4484


in = MapPartitionsRDD[17] at map at <console>:41


MapPartitionsRDD[17] at map at <console>:41

103820004 Michael Fu